In [1]:
!git clone https://github.com/YSdoubleM/Yooninahong.git

Cloning into 'Yooninahong'...
remote: Enumerating objects: 3693, done.
remote: Counting objects: 100% (3693/3693), done.
remote: Compressing objects: 100% (2207/2207), done.
remote: Total 3693 (delta 78), reused 3662 (delta 53), pack-reused 0
Receiving objects: 100% (3693/3693), 253.71 MiB | 13.85 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Checking out files: 100% (2674/2674), done.


In [ ]:
# train folder
# angle

In [2]:
def translate_coord(image,bbs):
  x_c = float(bbs[0])/image.shape[1]
  y_c = float(bbs[1])/image.shape[0]
  h = float(bbs[2])/image.shape[0]
  w = float(bbs[3])/image.shape[1]
  x_c = round(x_c,6)
  y_c = round(y_c,6)
  h = round(h,6)
  w = round(w,6)
  return x_c,y_c,h,w

In [3]:
def augmentation_angle(image_list,path,angle):
  i=0
  for file in image_list:
    image = imageio.imread(path+"/images/train/"+file)
    labelfile = file.replace('.jpg','')
    labelfile = labelfile.replace('.png','')
    labelfile = labelfile+'.txt'
    try:
      f = open(path+'/labels/train/'+labelfile)
    except:
      f = open(path+'/labels/train/'+labelfile.replace('.txt','')+'.jpg.txt')
    boundingboxes = f.readlines()
    new_bb = []
    for box in boundingboxes:
      info_list = box.split()

      class_name = info_list[0]
      x_c = float(info_list[1])*image.shape[1]
      y_c = float(info_list[2])*image.shape[0]
      h = float(info_list[3])*image.shape[0]
      w = float(info_list[4])*image.shape[1]

      new_bb.append(ia.BoundingBox(x_c,y_c,h,w))
    
    bbs = ia.BoundingBoxesOnImage(new_bb, shape = image.shape)

    seq = iaa.Sequential([iaa.Affine(rotate=(angle))])

    image_aug,bbs_aug = seq(image=image, bounding_boxes = bbs)
    bbs_aug=bbs_aug.to_xyxy_array()
    new_annotation = ""
    for box in bbs_aug:
      new_annotation += str(info_list[0])+' '+' '.join(map(str,translate_coord(image,box))) + "\n"
    
    
    try:
      image_aug = cv2.cvtColor(image_aug,cv2.COLOR_RGB2BGR)
      cv2.imwrite("/content/Yooninahong/dataset/images/train/"+str(angle)+str(i)+".jpg",image_aug)

      with open(path+"/labels/train/rotate_"+str(angle)+str(i)+".txt",'w') as f:
        f.write(new_annotation)
      i+=1
    except:
      pass

In [4]:
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
from google.colab.patches import cv2_imshow
from os import listdir

ia.seed(1)
path = '/content/Yooninahong/dataset'
image_list = listdir(path+"/images/train/")
print(len(image_list))

for angle in [60, -60, 90, -90]:
  augmentation_angle(image_list,path,angle)

1063


In [5]:
len(listdir('/content/Yooninahong/dataset/images/train'))

5275

In [6]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10582, done.
remote: Total 10582 (delta 0), reused 0 (delta 0), pack-reused 10582
Receiving objects: 100% (10582/10582), 10.72 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (7318/7318), done.


In [7]:
!pip install -r /content/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 4.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from glob import glob

train_img_list = glob('/content/Yooninahong/dataset/images/train/*.jpg')
val_img_list = glob('/content/Yooninahong/dataset/images/val/*.jpg')

In [9]:
with open('/content/Yooninahong/dataset/train.txt','w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/Yooninahong/dataset/val.txt','w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [10]:
import yaml
import os
data = dict()
data['train'] = '/content/Yooninahong/dataset/train.txt'
data['val'] = '/content/Yooninahong/dataset/val.txt'
data['nc'] = '7'
data['names'] = ['electricscooter','brailleblock','person','bike','motorcycle_F','bollard_short','bollard_long']

with open('/content/Yooninahong/dataset/data.yaml','w') as f:
  yaml.dump(data,f)
print(data)

{'train': '/content/Yooninahong/dataset/train.txt', 'val': '/content/Yooninahong/dataset/val.txt', 'nc': '7', 'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F', 'bollard_short', 'bollard_long']}


In [11]:
with open('/content/Yooninahong/dataset/data.yaml', 'r') as f:
  data = yaml.load(f,Loader = yaml.Loader)
print(data)

{'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F', 'bollard_short', 'bollard_long'], 'nc': '7', 'train': '/content/Yooninahong/dataset/train.txt', 'val': '/content/Yooninahong/dataset/val.txt'}


In [ ]:
!python /content/yolov5/train.py --img 640 --batch 32 --epoch 100 --data /content/Yooninahong/dataset/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5x.pt --name ynh_result

train: weights=yolov5x.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/Yooninahong/dataset/data.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=ynh_result, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-200-g9708cf5 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor